In [1]:
import pandas as pd
import os
import numpy as np
import sys
import cantera as ct

In [3]:
df = pd.read_csv('sim_data/rtol_1e-10_atol_1e-20_data.csv')
table_names = ['Ratio', 'SYNGAS Selec', 'SYNGAS Yield', 'CO Selectivity', 'CO % Yield', 'H2 Selectivity', 
               'H2 % Yield', 'CH4 Conversion', 'H2O+CO2 Selectivity', 'H2O+CO2 yield', 'Exit Temp', 'Peak Temp',
               'Dist to peak temp', 'O2 Conversion', 'Max CH4 Conv', 'Dist to 50 CH4 Conv', 'Half Temp Pos']
data_to_csv = []
for i in range(len(df)):
    sens_data = calc(df.iloc[i,1:], type='sens')
    data_to_csv.append(sens_data)
df_to_csv = pd.DataFrame(np.array(data_to_csv), columns=table_names)
df_to_csv.to_csv('sim_data/therm_sens_ref.csv')

In [2]:
def calc(data, type='sens'):
    """
    Calculate properties of interest from the raw data
    :param data: the data
    :param type: 'sens' for sensitivity analyses
                 'output' for saving the output csv
                 'ratio' for plotting
    :return:
    """
    ratio, ch4_in, ch4_out, co_out, h2_out, h2o_out, co2_out, reference_exit_temp, reference_peak_temp, reference_peak_temp_dist, reference_o2_conv, reference_max_ch4_conv, reference_dist_to_50_ch4_conv, sens_id= data

    ch4_depletion = ch4_in - ch4_out
    reference_ch4_conv = ch4_depletion / ch4_in 
    # negative sensitivity is higher selectivity
    reference_h2_sel = h2_out / (ch4_depletion * 2)  # Sensitivity definition 5: H2 selectivity
    if reference_h2_sel <= 0:
        reference_h2_sel = 1.0e-15  # selectivity can't be 0

    reference_co_sel = co_out / ch4_depletion  # Sensitivity definition 3: CO selectivity
    if reference_co_sel <= 0:
        reference_co_sel = 1.0e-15  # selectivity can't be 0

    reference_syngas_selectivity = reference_co_sel + reference_h2_sel  # Sensitivity definition 1: SYNGAS selectivity

    reference_syngas_yield = reference_syngas_selectivity * reference_ch4_conv  # Sensitivity definition 2: SYNGAS yield
    if reference_syngas_yield <= 0:
        reference_syngas_yield = 1.0e-15  # yield can't be 0

    reference_co_yield = co_out / ch4_in  # Sensitivity definition 4: CO % yield
    # reference_co_yield = reference_co_sel * reference_ch4_conv

    reference_h2_yield = h2_out / (2 * ch4_in)  # Sensitivity definition 6: H2 % yield
    # reference_h2_yield = reference_h2_sel * reference_ch4_conv

    # Sensitivity definition 8: H2O + CO2 selectivity
    reference_h2o_sel = h2o_out / (ch4_depletion * 2)
    reference_co2_sel = co2_out / ch4_depletion
    if reference_h2o_sel <= 0:
        reference_h2o_sel = 1.0e-15  # H2O selectivity can't be 0
    if reference_co2_sel <= 0:
        reference_co2_sel = 1.0e-15  # CO2 selectivity can't be 0
    reference_full_oxidation_selectivity = reference_h2o_sel + reference_co2_sel

    # Sensitivity definition 9: H2O + CO2 yield
    reference_full_oxidation_yield = reference_full_oxidation_selectivity * reference_ch4_conv
    
    sens_property = [ratio, reference_syngas_selectivity, reference_syngas_yield, reference_co_sel, reference_co_yield, reference_h2_sel, 
                     reference_h2_yield, reference_ch4_conv, reference_full_oxidation_selectivity, reference_full_oxidation_yield, 
                     reference_exit_temp, reference_peak_temp, reference_peak_temp_dist, reference_o2_conv, reference_max_ch4_conv, 
                     reference_dist_to_50_ch4_conv, sens_id]
    
    ref_data = [ratio, ch4_in, ch4_out, co_out, h2_out, h2o_out, co2_out, reference_exit_temp, reference_peak_temp, 
                reference_peak_temp_dist, reference_o2_conv, reference_max_ch4_conv, reference_dist_to_50_ch4_conv, sens_id]
    if type is 'sens':
        return sens_property
    elif type is 'ratio':
        return reference_co_sel, reference_h2_sel, reference_ch4_conv, reference_exit_temp, reference_o2_conv, reference_co2_sel, reference_h2o_sel
    elif type is 'gas_data':
        return ratio, reference
    else:
        return ref_data

In [5]:
x = pd.read_csv('sim_data/therm_sens_ref.csv')

In [6]:
x

,Unnamed: 0,Ratio,SYNGAS Selec,SYNGAS Yield,CO Selectivity,CO % Yield,H2 Selectivity,H2 % Yield,CH4 Conversion,H2O+CO2 Selectivity,H2O+CO2 yield,Exit Temp,Peak Temp,Dist to peak temp,O2 Conversion,Max CH4 Conv,Dist to 50 CH4 Conv,Half Temp Pos
0,0,0.6,1.005555,0.325391,0.360928,0.116794,0.644628,0.208597,0.323594,0.946349,0.306233,1909.003577,1909.003577,70.00,0.437927,-0.043935,10.87,1045.0
1,1,0.7,1.181959,0.375318,0.400448,0.127158,0.781512,0.248160,0.317539,0.808566,0.256751,1646.739416,1664.106650,13.73,0.448533,-0.047771,10.90,1045.0
2,2,0.8,1.323310,0.417508,0.451643,0.142495,0.871668,0.275014,0.315503,0.675285,0.213054,1406.561042,1489.441420,12.06,0.454896,-0.049903,10.92,1045.0
3,3,0.9,1.447275,0.441836,0.513060,0.156631,0.934215,0.285205,0.305288,0.552541,0.168684,1218.798831,1336.511668,11.28,0.444604,-0.048599,10.94,1045.0
4,4,1.0,1.530319,0.299905,0.549627,0.107713,0.980692,0.192192,0.195975,0.469646,0.092039,1143.189831,1218.616719,12.73,0.291793,-0.033057,11.28,1045.0
5,5,1.1,1.512677,0.173036,0.524153,0.059958,0.988525,0.113078,0.114391,0.487254,0.055737,1117.776513,1174.313331,13.70,0.188577,-0.022685,11.92,1045.0
6,6,1.2,1.485628,0.118496,0.494934,0.039477,0.990694,0.079020,0.079762,0.514271,0.041019,1103.886718,1148.712238,14.34,0.145819,-0.017503,12.64,1045.0
7,7,1.3,1.468808,0.088563,0.476948,0.028758,0.991860,0.059805,0.060296,0.531061,0.032021,1093.905088,1129.866126,14.87,0.120640,-0.014375,13.46,1045.0
8,8,1.4,1.460676,0.069876,0.468051,0.022391,0.992625,0.047485,0.047838,0.539165,0.025793,1085.860500,1114.642856,15.35,0.103567,-0.012217,14.41,1045.0
9,9,1.6,1.462495,0.048317,0.468906,0.015491,0.993589,0.032825,0.033037,0.537292,0.017751,1072.791181,1090.496662,16.26,0.081588,-0.010070,17.06,1045.0
